In [1]:
import pandas as pd
import numpy as np
from lsi_tagger import TagExtractor

%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\David\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
df = pd.read_csv('data/h&m_kaggle_products.csv')
df = df[~df.isnull().any(axis=1)]
print(df.shape)
df.head(2)

(105126, 25)


,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.


In [3]:
DOCUMENTS = df['detail_desc'].drop_duplicates().values

te = TagExtractor(word_count_min=5, word_length_min=2, num_lsi_topics=200, 
                  bigram_kwargs={'bigrams_pmi_min_value':1, 'bigrams_min_freq':200})
te.fit(DOCUMENTS)

Filtering tokens: 100%|████████████████████████████████████████████████████████| 43404/43404 [00:10<00:00, 3974.98it/s]
Making bigrams: 43404it [00:00, 55542.32it/s]
Filtering by word count: 100%|████████████████████████████████████████████████| 43404/43404 [00:00<00:00, 66273.54it/s]
2022-04-20 22:01:06 INFO     Training TF-IDF...
2022-04-20 22:01:06 INFO     adding document #0 to Dictionary(0 unique tokens: [])
2022-04-20 22:01:06 INFO     adding document #10000 to Dictionary(1700 unique tokens: ['jersey', 'jersey top', 'narrow', 'narrow shoulder', 'shoulder']...)
2022-04-20 22:01:07 INFO     adding document #20000 to Dictionary(1869 unique tokens: ['jersey', 'jersey top', 'narrow', 'narrow shoulder', 'shoulder']...)
2022-04-20 22:01:08 INFO     adding document #30000 to Dictionary(1979 unique tokens: ['jersey', 'jersey top', 'narrow', 'narrow shoulder', 'shoulder']...)
2022-04-20 22:01:08 INFO     adding document #40000 to Dictionary(2035 unique tokens: ['jersey', 'jersey top', 'nar

In [4]:
len(te.tc.bigrams_dict)

200

In [5]:
tags = te.transform(input_document='Jersey top with narrow shoulder straps.',
                    candidate_documents=[
                        'Tights with built-in support to lift the bottom. Black in 30 denier and light amber in 15 denier.',
                        'Semi shiny tights that shape the tummy, thighs and calves while also encouraging blood circulation in the legs. Elasticated waist.',
                        'Jersey top with narrow shoulder straps.',
                        'uygnhjkl'
                    ], 
                    n_input_tags=10, n_candidate_tags=10, n_adjacent_tags=10)

tags

([('jersey top', 1),
  ('narrow shoulder', 1),
  ('shoulder', 1),
  ('shoulder straps', 1),
  ('narrow', 1),
  ('straps', 1),
  ('top', 1),
  ('jersey', 1)],
 [{},
  {},
  {'jersey top': 0.8355987736033494,
   'narrow shoulder': 0.7715389780639272,
   'shoulder': 0.6346510475397362,
   'shoulder straps': 0.6062105644915914,
   'narrow': 0.5855368140516707,
   'straps': 0.5750784223206391,
   'top': 0.3979580132020514,
   'jersey': 0.3564798515181357},
  {}],
 ['join',
  'knots',
  'stitched',
  'cold',
  'swimsuits',
  'cross',
  'tankini',
  'fully-lined',
  'stripe-print',
  'modal'])

In [6]:
new_candidate_ranking = te.rank(candidate_tags=tags[1], selected_tags=['jersey'])
new_candidate_ranking

[2, 0, 1, 3]

In [7]:
te.save()
te = TagExtractor()
te.load()

In [8]:
tags = te.transform(input_document='Jersey top with narrow shoulder straps.',
                    candidate_documents=[
                        'Tights with built-in support to lift the bottom. Black in 30 denier and light amber in 15 denier.',
                        'Semi shiny tights that shape the tummy, thighs and calves while also encouraging blood circulation in the legs. Elasticated waist.',
                        'Jersey top with narrow shoulder straps.',
                        'uygnhjkl'
                    ], 
                    n_input_tags=10, n_candidate_tags=10, n_adjacent_tags=10)

tags

([('jersey top', 1),
  ('narrow shoulder', 1),
  ('shoulder', 1),
  ('shoulder straps', 1),
  ('narrow', 1),
  ('straps', 1),
  ('top', 1),
  ('jersey', 1)],
 [{},
  {},
  {'jersey top': 0.8355987736033494,
   'narrow shoulder': 0.7715389780639272,
   'shoulder': 0.6346510475397362,
   'shoulder straps': 0.6062105644915914,
   'narrow': 0.5855368140516707,
   'straps': 0.5750784223206391,
   'top': 0.3979580132020514,
   'jersey': 0.3564798515181357},
  {}],
 ['join',
  'knots',
  'stitched',
  'cold',
  'swimsuits',
  'cross',
  'tankini',
  'fully-lined',
  'stripe-print',
  'modal'])

In [9]:
new_candidate_ranking = te.rank(candidate_tags=tags[1], selected_tags=['jersey'])
new_candidate_ranking

[2, 0, 1, 3]

In [10]:
from tqdm import tqdm

class ContentRecommender:
    def __init__(self):
        lsi_vecs = np.array([te._lsi_corpus2vec(cl) for cl in 
                             tqdm(te.corpus_lsi, desc='Creating LSI Vectors', position=0)])
        self.lsi_vecs_normed = lsi_vecs/np.linalg.norm(lsi_vecs, axis=1)[:,None]
    
    def _show(self, candidate_documents, candidate_tags):
        print("#########################")
        print("######### INPUTS ########")
        print("#########################\n")
        print(f"Input Document: {self.input_document}")
        print(f"\nAdjacent Tags: {self.adjacent_tags}")
        print(f"\nInput Tags: {self.input_tags}")
        
        print("\n#########################")
        print("#### RECOMMENDATIONS ####")
        print("#########################\n")
        for n, candidate_doc in enumerate(candidate_documents):
            print(f"{n+1}. {candidate_doc}")
            display(candidate_tags[n])
            print('-'*150)
            if (n+1) == self.show_n_recs:
                break
                
    def _get_content_recommendations(self, input_document):
        _, input_corpus_lsi = te._get_vector_representations(input_document)
        input_lsi_vec = te._lsi_corpus2vec(input_corpus_lsi)
        input_lsi_vec_normed = input_lsi_vec/np.linalg.norm(input_lsi_vec)
        sims = input_lsi_vec_normed.dot(self.lsi_vecs_normed.T)
        top_inds = np.argsort(sims)[::-1][:(self.internal_n_recs+1)]
        self.candidate_documents = [DOCUMENTS[ti] for ti in top_inds if DOCUMENTS[ti]!=input_document]
        tags = te.transform(input_document=input_document,
                            candidate_documents=self.candidate_documents, 
                            n_input_tags=self.n_input_tags, 
                            n_candidate_tags=self.n_candidate_tags,
                            n_adjacent_tags=self.n_adjacent_tags)
        self.input_tags, self.candidate_tags, self.adjacent_tags = tags
    
    def initial_recommendations(self, input_document, 
                                internal_n_recs=50, show_n_recs=5,
                                n_input_tags=10, n_candidate_tags=5,
                                n_adjacent_tags=10):
        self.__dict__.update(locals())
        self._get_content_recommendations(self.input_document)
        self._show(self.candidate_documents, self.candidate_tags)
        
    def rerank(self, selected_tags):
        ranking = te.rank(candidate_tags=self.candidate_tags, 
                          selected_tags=selected_tags)
        candidate_documents = np.array(self.candidate_documents)[ranking]
        candidate_tags = np.array(self.candidate_tags)[ranking]
        self._show(candidate_documents, candidate_tags)

In [11]:
recommender = ContentRecommender()

Creating LSI Vectors: 100%|████████████████████████████████████████████████████| 43404/43404 [00:22<00:00, 1948.76it/s]


In [12]:
# Use an existing product description as the input, to find similar products
recommender.initial_recommendations(input_document=DOCUMENTS[10000], 
                                    internal_n_recs=50, show_n_recs=5,
                                    n_input_tags=10, n_candidate_tags=5, 
                                    n_adjacent_tags=10)

#########################
######### INPUTS ########
#########################

Input Document: Straight-cut, linen-blend jacket in a patterned weave with a shawl collar, padded shoulders and long sleeves with decorative buttons at the cuffs. Buttons at the front and front pockets. Lined.

Adjacent Tags: ['two-button', 'lapel', 'taper', 'single-breasted', 'stitching', 'buttonhole', 'scallop-edged', 'narrower', 'combined', 'creates']

Input Tags: [('buttons', 49), ('decorative', 45), ('jacket', 44), ('decorative buttons', 42), ('straight-cut', 26), ('front', 21), ('front pockets', 7), ('long sleeves', 7), ('patterned', 3), ('long', 3)]

#########################
#### RECOMMENDATIONS ####
#########################

1. Straight-cut, double-breasted jacket in a patterned weave with notch lapels and covered buttons at the front. Long sleeves with decorative buttons at the cuffs and welt front pockets with a flap. Lined.


{'buttons': 0.6086969549677756,
 'straight-cut': 0.5176992024999797,
 'decorative': 0.44666291142899206,
 'jacket': 0.39151369258301355,
 'decorative buttons': 0.36712758040560545}

------------------------------------------------------------------------------------------------------------------------------------------------------
2. Single-breasted jacket in a viscose weave with peak lapels, jetted front pockets and decorative buttons at the cuffs. Lined.


{'buttons': 0.5141416383651543,
 'decorative': 0.48783265861733266,
 'jacket': 0.3944050077881121,
 'decorative buttons': 0.3752650265443899,
 'front': 0.3038119973683874}

------------------------------------------------------------------------------------------------------------------------------------------------------
3. Straight-cut jacket in a patterned linen and viscose weave with notch lapels, buttons at the front, welt front pockets and decorative buttons at the cuffs. Partly lined.


...

------------------------------------------------------------------------------------------------------------------------------------------------------
4. Straight-cut, double-breasted jacket in woven fabric with peak lapels, welt front pockets and decorative buttons at the cuffs. Lined.


{'straight-cut': 0.5408663093904469,
 'buttons': 0.5038833615901823,
 'decorative': 0.4672735974751826,
 'jacket': 0.39448768929421774,
 'decorative buttons': 0.35694757281525025}

------------------------------------------------------------------------------------------------------------------------------------------------------
5. Double-breasted jacket in a striped weave with peak lapels, jetted front pockets and decorative buttons at the cuffs. Lined.


{'buttons': 0.48842789629467764,
 'decorative': 0.47891939847714887,
 'jacket': 0.36218227428102384,
 'decorative buttons': 0.34518308921777097,
 'front': 0.29076341695990715}

------------------------------------------------------------------------------------------------------------------------------------------------------


In [13]:
# Re-rank product recommendations with emphasis on a few input tags
recommender.rerank(selected_tags=['patterned'])

#########################
######### INPUTS ########
#########################

Input Document: Straight-cut, linen-blend jacket in a patterned weave with a shawl collar, padded shoulders and long sleeves with decorative buttons at the cuffs. Buttons at the front and front pockets. Lined.

Adjacent Tags: ['two-button', 'lapel', 'taper', 'single-breasted', 'stitching', 'buttonhole', 'scallop-edged', 'narrower', 'combined', 'creates']

Input Tags: [('buttons', 49), ('decorative', 45), ('jacket', 44), ('decorative buttons', 42), ('straight-cut', 26), ('front', 21), ('front pockets', 7), ('long sleeves', 7), ('patterned', 3), ('long', 3)]

#########################
#### RECOMMENDATIONS ####
#########################

1. Straight-cut shirt in a soft, patterned viscose weave with a collar and buttons down the front. Long sleeves with narrow, buttoned cuffs, and a rounded hem.


{'straight-cut': 0.6344095605564022,
 'buttons': 0.523922469564337,
 'patterned': 0.4693668942986713,
 'long sleeves': 0.36422420439827247,
 'weave': 0.33389358055019036}

------------------------------------------------------------------------------------------------------------------------------------------------------
2. Short, straight-cut coat in a patterned weave with a zip down the front, long sleeves and zipped front pockets with a fake flap. Lined.


{'straight-cut': 0.616711410463947,
 'patterned': 0.4327267917499439,
 'front': 0.32006705153595605,
 'front pockets': 0.3177362230233681,
 'long sleeves': 0.31441190773583955}

------------------------------------------------------------------------------------------------------------------------------------------------------
3. Jacket in a linen and cotton weave with notch lapels, buttons at the front, a chest pocket and flap front pockets. Decorative buttons at the cuffs and a single back vent. Patterned lining.


...

------------------------------------------------------------------------------------------------------------------------------------------------------
4. Straight-cut, double-breasted jacket in a patterned weave with notch lapels and covered buttons at the front. Long sleeves with decorative buttons at the cuffs and welt front pockets with a flap. Lined.


{'buttons': 0.6086969549677756,
 'straight-cut': 0.5176992024999797,
 'decorative': 0.44666291142899206,
 'jacket': 0.39151369258301355,
 'decorative buttons': 0.36712758040560545}

------------------------------------------------------------------------------------------------------------------------------------------------------
5. Single-breasted jacket in a viscose weave with peak lapels, jetted front pockets and decorative buttons at the cuffs. Lined.


{'buttons': 0.5141416383651543,
 'decorative': 0.48783265861733266,
 'jacket': 0.3944050077881121,
 'decorative buttons': 0.3752650265443899,
 'front': 0.3038119973683874}

------------------------------------------------------------------------------------------------------------------------------------------------------


In [14]:
# Use free-text to find products that match the description best
recommender.initial_recommendations(input_document='leather gloves', 
                                    internal_n_recs=50, show_n_recs=5,
                                    n_input_tags=10, n_candidate_tags=5, 
                                    n_adjacent_tags=10)

#########################
######### INPUTS ########
#########################

Input Document: leather gloves

Adjacent Tags: ['uppers', 'fingerless', 'finger', 'touchscreen-compatible', 'index', 'separately', 'thumb', 'palms', 'reveal', 'material']

Input Tags: [('leather', 39), ('gloves', 28)]

#########################
#### RECOMMENDATIONS ####
#########################

1. Gloves in soft, supple leather. Lined.


{'leather': 0.2655559532547105, 'gloves': 0.1512622589364015}

------------------------------------------------------------------------------------------------------------------------------------------------------
2. Knitted gloves with imitation leather uppers and palms, and foldover cuffs. Fleece lining.


{'leather': 0.32525198340333217, 'gloves': 0.18711892751843423}

------------------------------------------------------------------------------------------------------------------------------------------------------
3. Gloves in soft, supple imitation leather. Lined.


...

------------------------------------------------------------------------------------------------------------------------------------------------------
4. Gloves in soft leather with a wool-blend lining.


{'leather': 0.25396827454078963, 'gloves': 0.15886387154538423}

------------------------------------------------------------------------------------------------------------------------------------------------------
5. Gloves in soft, supple leather with a zip at the top. Lined.


{'leather': 0.26481560983697017, 'gloves': 0.14917538079734338}

------------------------------------------------------------------------------------------------------------------------------------------------------


In [15]:
# Use adjacent tags to make a different query
recommender.initial_recommendations(input_document='touchscreen-compatible leather gloves', 
                                    internal_n_recs=50, show_n_recs=5,
                                    n_input_tags=10, n_candidate_tags=5, 
                                    n_adjacent_tags=10)

#########################
######### INPUTS ########
#########################

Input Document: touchscreen-compatible leather gloves

Adjacent Tags: ['uppers', 'fingerless', 'finger', 'index', 'thumb', 'separately', 'palms', 'material', 'reveal', 'wrists']

Input Tags: [('leather', 34), ('gloves', 34), ('touchscreen-compatible', 4)]

#########################
#### RECOMMENDATIONS ####
#########################

1. Gloves in soft, supple leather. Lined.


{'leather': 0.2073988527845495, 'gloves': 0.12352022669882463}

------------------------------------------------------------------------------------------------------------------------------------------------------
2. Knitted gloves with imitation leather uppers and palms, and foldover cuffs. Fleece lining.


{'leather': 0.2670948829331712, 'gloves': 0.15937689528085738}

------------------------------------------------------------------------------------------------------------------------------------------------------
3. Gloves in soft, supple imitation leather. Lined.


...

------------------------------------------------------------------------------------------------------------------------------------------------------
4. Gloves in soft leather with a wool-blend lining.


{'leather': 0.1958111740706286, 'gloves': 0.13112183930780735}

------------------------------------------------------------------------------------------------------------------------------------------------------
5. Gloves in soft, supple leather with a zip at the top. Lined.


{'leather': 0.20665850936680918, 'gloves': 0.1214333485597665}

------------------------------------------------------------------------------------------------------------------------------------------------------


In [16]:
# Re-rank product recommendations based on an input tag
recommender.rerank(selected_tags=['touchscreen-compatible'])

#########################
######### INPUTS ########
#########################

Input Document: touchscreen-compatible leather gloves

Adjacent Tags: ['uppers', 'fingerless', 'finger', 'index', 'thumb', 'separately', 'palms', 'material', 'reveal', 'wrists']

Input Tags: [('leather', 34), ('gloves', 34), ('touchscreen-compatible', 4)]

#########################
#### RECOMMENDATIONS ####
#########################

1. Water-repellent, softshell gloves with a soft fleece lining and reinforced palms in imitation leather. Elastication at the wrists, reflective details and a plastic hook-and-eye fastening to keep the gloves together. The tops of the thumb and index finger are made from touchscreen-compatible material.


{'leather': 0.19472717214131188,
 'gloves': 0.15001561130385516,
 'touchscreen-compatible': 0.012376869801815151}

------------------------------------------------------------------------------------------------------------------------------------------------------
2. Leather gloves with the tops of the thumb and index finger in touchscreen-compatible material. Lined.


{'leather': 0.1561100997109584,
 'gloves': 0.10653147568587917,
 'touchscreen-compatible': 0.010008457553765643}

------------------------------------------------------------------------------------------------------------------------------------------------------
3. Leather gloves. The tops of the thumb, index finger and middle finger have been treated to make them touchscreen-compatible. Lined.


...

------------------------------------------------------------------------------------------------------------------------------------------------------
4. Fine-knit gloves with touchscreen-compatible material on the thumb, index and middle finger.


{'gloves': 0.10371584273337257, 'touchscreen-compatible': 0.007186697664351248}

------------------------------------------------------------------------------------------------------------------------------------------------------
5. Gloves in soft, supple leather. Lined.


{'leather': 0.2073988527845495, 'gloves': 0.12352022669882463}

------------------------------------------------------------------------------------------------------------------------------------------------------
